# #. Group Units

A Group is a collection of Group Units. A Group Unit consists of:
- **Nonce**: A unique identifier for the Group Unit.  Also, declares the Group Unit's Hierarchical Deterministic (HD) path.
- **Owner**: The owner of the Group Unit.  Generally, the owner is the Group's creator, but alternative ownership models are possible.
- **Credential**: A credential that is used to authenticate the Group Unit and allow access to the Group Unit's data.
- **Data**: The data that is stored in the Group Unit.

In [1]:
import sys

sys.path.append("../../forme-groups-python-3-12/")

## Controller

The Controller is the entity that creates the Group.  The Controller is responsible for:
- Creating the Group.
- Creating the Group Units.

In [2]:
from src.groups.controller import Controller
from src.groups.unit import Data

# Create a Controller with a fresh group unit pool
controller = Controller()

# Create some data to store in the Group
data = Data((1, 2, 3))

# Create a GroupUnit and store the data in it
group_unit = controller._create_group_unit(data)

print(group_unit)

nonce: 1, owner: , credential: , data: entry: (1, 2, 3), schema: None


## Create Sub Units from a schema



In [6]:
from src.groups.base import SchemaEntry, BaseSchema

# Create a SchemaEntry for the GroupUnit
schema_entry = SchemaEntry("name", "str")
schema_entry2 = SchemaEntry("age", "int")
schema_for_sub_units = BaseSchema((schema_entry, schema_entry2))

# Create the data to store in the GroupUnit
data = Data(('a', 'b', 'c'), schema_for_sub_units)

# Create a GroupUnit and store the data in it
group_unit = controller._create_group_unit(data)

# Create the data with the proper schema
sub_unit_data = Data(("Fred", 12))

sub_group_unit = controller._create_group_unit(sub_unit_data, True)

print(sub_group_unit)


ValueError: GroupUnit with package_hash 19040bd68890ac87e845aa5adf17a2f8b9d4dce83ead0f7ca279b61da0caaaa7 already exists in the Pool